In [2]:
!pip install eli5

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import FunctionTransformer
import eli5
from eli5.sklearn import PermutationImportance
from nltk.corpus import stopwords
from string import punctuation
from sklearn.ensemble import VotingClassifier 
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier

In [5]:
from google.colab import drive 
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [6]:
data = pd.read_csv('/gdrive/My Drive/exam/jigsaw-toxic-comment-train.csv.zip')

## 1. Описательный анализ данных с привязкой к целевой переменной (toxic) и 2 другим дополнительным колонкам (как минимум 5 статистик на каждую группу) - 1.5 балл

In [ ]:
data.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [9]:
len(data[data['toxic']==1])

21384

In [10]:
len(data[data['toxic']==0])

202165

In [ ]:
data.shape

(223549, 8)

In [ ]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

In [ ]:
stops = stopwords.words('english')

In [ ]:
data['text_len'] = data['comment_text'].apply(len) #длина текста
data['text_len_tokens'] = data['comment_text'].apply(lambda x: len(x.split())) #количество токенов
data['number_of_caps'] = data['comment_text'].apply(lambda x: len([ch for ch in x if ch.isupper()])) #количество слов с заглавной буквы
data['number_of_nonalphanum'] = data['comment_text'].apply(lambda x: len([ch for ch in x if not ch.isalnum()])) #количество  не буквенно-цифровых символов
data['number_of_lows'] = data['comment_text'].apply(lambda x: len([ch for ch in x if ch.islower()])) #количество строчных букв
data['number_of_digits'] = data['comment_text'].apply(lambda x: len([ch for ch in x if ch.isdigit()])) #количество цифр
data['number_of_ascii'] = data['comment_text'].apply(lambda x: len([ch for ch in x if is_ascii(ch)])) #количество ascii символов
data['number_of_nonascii'] = data['comment_text'].apply(lambda x: len([ch for ch in x if not is_ascii(ch)])) #количество non-ascii символов
data['number_of_punctuation'] = data['comment_text'].apply(lambda x: len([ch for ch in x if ch in punctuation])) #количество знаков пунктуации
data['number_of_stops'] = data['comment_text'].apply(lambda x: len([ch for ch in x if ch in stops])) #количество стоп-слов

In [ ]:
data.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,264,43,17,52,186,9,264,0,10,84
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,112,17,8,29,65,10,112,0,12,29
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,233,42,4,47,182,0,233,0,6,99


### Отдельные колонки

### Toxic/Non-toxic

In [ ]:
data.groupby('toxic')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
toxic,,,,,,,,,,
0,402.691178,68.415161,14.484189,89.072456,294.922009,3.248698,400.972142,1.719036,16.974704,141.799609
1,280.604097,48.573466,44.689675,65.266741,168.618921,1.995885,280.409512,0.194585,14.257202,80.432941


1. Средняя длина нетоксичного комментария выше, чем у токсичного. 
2. Среднее количество слов у нетоксичных текстов также больше. 
3. Среднее количество используемых заглавных букв у токсичных сообщений в несколько рах выше, чем у нетоксичных.
4. Среднее количество  не буквенно-цифровых символов в нетоксичных комментариях выше.
5. Количество строчных букв в токсичных комментариях меньше, чем в нетоксичных.
6. Цифр в нетоксичных комментариях также в среднем больше. 
7. Ascii-символов в нетоксичных комментариях больше.
8. Не-ascii символов в целом по комментариям практически нет, но если они встречаются, то чаще в нетоксичных комментариях.
9. Оба типа комментариев имеют практически одинаковое среднее количество знаков препинания. В нетоксичных комментариях их чуть больше. 
10. Используемых стоп-слов в нетоксичных комментариях гораздо больше, чем в токсичных.

### Severe_toxic/Non-severe_toxic

In [ ]:
data.groupby('severe_toxic')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
severe_toxic,,,,,,,,,,
0,389.999386,66.358270,15.988131,86.500630,283.491852,3.136258,388.413228,1.586158,16.591014,136.307514
1,505.456677,84.462793,173.841998,120.072375,209.248726,2.293068,505.345566,0.111111,30.690112,93.232926


1. Средняя длина сильно токсичных комментариев гораздо больше. 
2. Среднее количество слов в сильно токсичных комментариях выше.
3. Количество заглавных букв в сильно токсичных текстах во много раз превышает количество в обычных комментариях.
4. Среднее количество не буквенно-цифровых символов в токсичных комментариях выше.
5. Строчных букв в обычных комментариях больше. 
6. В обоих типах комментариев цифр мало, но в обычных комментариях в среднем чуть больше.
7. Ascii-символов больше в сильно токсичных комментариях.
8. Не-ascii символы практически отсутствуют в обоих типах. В обычных чуть больше.
9. Количество знаков пунктуации в среднем почти в 2 раза больше в токсичных комментариях.
10. Количество стоп-слов в обычных комментариях в среднем больше. 

### Obscene/Non-obscene

In [ ]:
data.groupby('obscene')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
obscene,,,,,,,,,,
0,397.332876,67.559707,15.299590,88.032052,289.883993,3.193029,395.678353,1.654523,16.850087,139.417163
1,280.951730,48.362109,53.490033,65.257743,160.178748,2.011367,280.794481,0.157249,14.358072,75.193822


1. Средняя длина грубых комментарией намного меньше, чем обычных.
2. Количество слов в грубых текстах также меньше в среднем на 19.
3. Заглавных букв в грубых комментариях во много раз больше.
4. Среднее количество не буквенно-цифровых символов в грубых комментариях меньше.
5. Негрубые тексты содержат гораздо больше строчных букв. 
6. Оба типа комментариев содержат мало цифр. В негрубых их в среднем на 1 больше.
7. Количество ascii-символов практически полностью совпадает со средней длиной комментариев в обих типах.
8. В негрубых в среднем встречается 1.6 не ascii-символов, что чуть больше, чем в грубых.
9. Количество знаков пунктуации практически одинаковое, но в негрубых их на 2 больше, чем в грубых.
10. В грубых комментариях используется меньше стоп-слов, чем в негрубых почти в 2 раза.

### Threat/Non-threat

In [ ]:
data.groupby('threat')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
threat,,,,,,,,,,
0,391.241807,66.547128,17.149430,86.796312,283.283766,3.134856,389.665157,1.576649,16.686637,136.141282
1,316.910015,56.825835,89.866473,86.460087,139.383164,1.188679,316.448476,0.461538,25.809869,67.416546


1. Длина текстов с угрозой меньше, чем текстов без угрозы.
2. Количество слов в текстах без угрозы на 10 больше. 
3. Количество заглавных букв в текстах с угрозой во много раз больше, чем в текстах без.
4. Среднее количество не буквенно-цифровых символов одинаково.
5. Количество строчных букв в текстах без угрозы больше, чем в 2 раза.
6. Количество цифр в обоих типах мало, но в текстах без угрозы их в среднем чуть больше.
7. Количество ascii-символов практически полностью совпадает со средней длиной комментариев в обих типах.
8. Не ascii-символы практически отсутствуют в обоих типах. В текстах без угрозы в среднем их 1.6.
9. Знаки пунктуации присутствуют в обоих типах комментариев и их количество почти одинаковое.
10. Количество использованных стоп-слов больше в текстах без угрозы в 2 раза. 

### Insult/Non-insult

In [ ]:
data.groupby('insult')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
insult,,,,,,,,,,
0,397.340234,67.544536,15.573931,88.053302,289.597522,3.194615,395.690669,1.649565,16.879710,139.233245
1,272.206476,47.227176,51.163393,63.174452,155.965057,1.894197,272.066879,0.139597,13.617569,73.897381


1. Средняя длина комментария с оскорблением меньше, чем обычный комментарий.
2. В обычном комментарии в среднем на 20 слов больше, чем в тексте с оскорблением.
3. Количество заглавных букв в тексте с оскорблением в несколько раз больше.
4. Среднее количество не буквенно-цифровых символов в обычных комментариях больше, чем в оскорбительных.
5. Количество строчных букв почти в 2 раза больше в комментариях без оскорбления.
6. Количество цифр в обоих типах мало, но в текстах без оскорбления их в среднем чуть больше.
7. Количество ascii-символов практически полностью совпадает со средней длиной комментариев в обих типах.
8. Не ascii-символы практически отсутствуют в обоих типах. В текстах без оскорбления в среднем их 1.6.
9. Знаки пунктуации присутствуют в обоих типах комментариев и их количество почти одинаковое.
10. Количество использованных стоп-слов больше в текстах без оскорбления в 2 раза.

### Identity_hate/Non-identity_hate

In [ ]:
data.groupby('identity_hate')['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,text_len,text_len_tokens,number_of_caps,number_of_nonalphanum,number_of_lows,number_of_digits,number_of_ascii,number_of_nonascii,number_of_punctuation,number_of_stops
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
identity_hate,,,,,,,,,,
0,391.744793,66.651428,16.774599,86.960259,283.989184,3.137812,390.158234,1.586559,16.740187,136.492779
1,314.438829,52.473784,80.022201,69.538498,162.665092,2.192253,314.261691,0.177137,14.054795,77.008503


1. Средняя длина комментария без ненависти личности больше на примерно 75 слов.
2. Обычные комментарии в среднем имеют больше слов.
3. Количество заглавных букв в тексте с ненавистью личности в несколько раз больше.
4. Среднее количество не буквенно-цифровых символов в обычных комментариях больше. 
5. Количество строчных букв почти в 2 раза больше в комментариях без ненависти.
6. Количество цифр в обоих типах мало, но в текстах без оскорбления их в среднем чуть больше.
7. Количество ascii-символов практически полностью совпадает со средней длиной комментариев в обих типах.
8. Не ascii-символы практически отсутствуют в обоих типах. В текстах без ненависти в среднем их 1.58.
9. Знаки пунктуации присутствуют в обоих типах комментариев и их количество почти одинаковое.
10. Количество использованных стоп-слов больше в текстах без ненависти меньше, чем в 2 раза.

### Привязка к другим колонкам

### Toxic/Insult

In [ ]:
data.groupby(['toxic', 'insult'])['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


text_len text_len_tokens  ... number_of_punctuation number_of_stops
                    mean            mean  ...                  mean            mean
toxic insult                              ...                                      
0     0       402.813001       68.433497  ...             16.978293      141.864081
      1       362.961165       62.435275  ...             15.804207      120.773463
1     0       294.234997       50.796784  ...             15.022434       89.669097
      1       266.957889       46.347651  ...             13.491110       71.186412

[4 rows x 10 columns]

1. Non-toxic/non-insult имеют самую большую среднюю длину. В non-toxic/insult комментариях длина текста больше, чем просто в toxic/non-insult. Toxic/insult в свою очередь имеют самые короткие тексты.
2. Non-toxic/non-insult имеют самое большое количество слов. В insult комментариях слов в среднем на 12 больше, чем просто в toxic/non-insult. Toxic/insult в свою очередь имеют меньше всего слов.
3. Заглавных букв больше всего в комментариях имеющие обе метки toxic/insult. Меньше всего в обычных комментариях без меток. Токсичные комментарии имеют больше заглавных букв, чем оскорбительные.
4. Среднее количество не буквенно-цифровых символов в обычных комментариях больше. Toxic/non-insult комментарии имеют меньше данных символов, чем non-toxic/insult. 
5. Количество строчных букв больше всего в комментариях без меток. Меньше всего строчных букв в комментариях с обеими метками toxic и insult. 
6. В комментариях всех типов используется мало цифр. Больше всего в текстах non-toxic/insult, меньше всего в toxic/insult. 
7. Количество ascii-символов практически полностью совпадает со средней длиной комментариев во всех типах.
8. Не ascii-символы практически отсутствуют в данных типах.
9. Знаков пунктуации больше всего в обычных комментариях. В комментариях toxic/insult их наименьшее количество.
10. Больше всего стоп-слов в обычных комментариях. Меньше всего в toxic/insult. В toxic/non-insult их меньше, чем в non-toxic/insult.

### Toxic/Obscene

In [ ]:
data.groupby(['toxic', 'obscene'])['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


text_len text_len_tokens  ... number_of_punctuation number_of_stops
                     mean            mean  ...                  mean            mean
toxic obscene                              ...                                      
0     0        402.480948       68.373545  ...             16.961355      141.744579
      1        474.761905       82.681973  ...             21.551020      160.664966
1     0        291.786412       50.874288  ...             14.568857       91.700163
      1        271.086738       46.615218  ...             13.991949       70.843317

[4 rows x 10 columns]

1. Non-toxic/obscene комментарии являются самыми длинными. Toxic/obscene имеют самую маленькую длину.
2. Больше всего слов в грубых комментариях non-toxic/obscene. Меньше всего в toxic/obscene вместе.
3. Наибольшее количество заглавных букв наблюдается в toxic/obscene комментариях. В нормальных комментариях их меньше всего. Грубые и токсичные комментарии отдельно имеют примерно одинаковое количество заглавных букв.
4. Среднее количество не буквенно-цифровых символов в грубых комментариях больше. Toxic/obscene комментарии имеют меньше всего данных символов.
5. Больше всего строчных букв в грубых комментариях non-toxic/obscene. Toxic/obscene имеют меньше всего данных букв.
6. В комментариях всех типов используется мало цифр. Больше всего в текстах obscene, меньше всего в toxic/obscene.
7. Количество ascii-символов практически полностью совпадает со средней длиной комментариев в обих типах.
8. Не ascii-символы практически отсутствуют в данных типах.
9. Больше всего знаков пунктуации в грубых комментариях non-toxic/obscene.
10. Non-toxic/obscene грубые комментарии имеют больше всего стоп-слов. Меньше всего в toxic/obscene.

### Toxic/Threat

In [ ]:
data.groupby(['toxic', 'threat'])['text_len', 'text_len_tokens', 'number_of_caps', 'number_of_nonalphanum', 'number_of_lows', 'number_of_digits', 
                      'number_of_ascii', 'number_of_nonascii', 'number_of_punctuation', 'number_of_stops'].agg(['mean'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


text_len text_len_tokens  ... number_of_punctuation number_of_stops
                    mean            mean  ...                  mean            mean
toxic threat                              ...                                      
0     0       402.693648       68.415233  ...             16.975268      141.805764
      1       388.428571       68.000000  ...             13.714286      106.257143
1     0       279.579450       48.331983  ...             13.872311       80.909165
      1       313.082569       56.227829  ...             26.457187       65.337920

[4 rows x 10 columns]

1. Обычные комментарии самые длинные. Самыми короткими являются комментарии типа toxic/non-threat. В non-toxic/threat также наблюдается большая средняя длина. 
2. Обычные комментарии и non-toxic/threat имеют одинаковое среднее количество слов. Меньше всего слов в toxic/non-threat комментариях. 
3. Заглавных букв больше всего в toxic/threat комментариях. Меньше всего в обычных. Тексты с угрозой имеют больше заглавных букв, чем токсичные.
4. Среднее количество не буквенно-цифровых символов незначительно отличается. В toxic/non-threat комментариях их меньше всего. В остальных типах их примерно одинаково.
5. Больше всего строчных букв в обычных комментариях. Меньше всего в toxic/threat. Non-toxic/threat имеют больше строчных букв, чем toxic/non threat. 
6. В комментариях всех типов используется мало цифр. Больше всего в текстах threat, меньше всего в toxic/threat.
7. Количество ascii-символов практически полностью совпадает со средней длиной комментариев в обих типах.
8. Не ascii-символы практически отсутствуют в данных типах.
9. Больше всего знаков пунктуации в текстах toxic/threat. В toxic/non-threat и non-toxic/threat их примерно одинаково.
10. Наибольшее количество стоп-слов наблюдается в текстах комментариев типа non-toxic/non-threat. Меньше всего в toxic/threat. В комментариях non-toxic/threat их больше, чем в toxic/non-threat. 

## 2. Бейзлайн модель из sklearn (векторайзер + модель) с отбором признаков (через l1 регуляризацию, на глаз через анализ важных параметров или через permutation importance) - 2 балл

### Бейзлайн

In [28]:
X = data['comment_text']
y = data['toxic']

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [30]:
count_vect = CountVectorizer()

In [31]:
X_train = count_vect.fit_transform(X_train)
X_train.shape

(149777, 207901)

In [32]:
X_test = count_vect.transform(X_test)
X_test.shape

(73772, 207901)

In [33]:
clf = SGDClassifier().fit(X_train, y_train)

In [34]:
y_pred = clf.predict(X_test)

In [35]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     66765
           1       0.82      0.60      0.69      7007

    accuracy                           0.95     73772
   macro avg       0.89      0.79      0.83     73772
weighted avg       0.95      0.95      0.95     73772



### Top features Eli5

In [17]:
eli5.formatters.as_dataframe.explain_weights_df(clf)


,target,feature,weight
0,1,x70089,2.211976
1,1,x84835,2.087682
2,1,x64497,2.057759
3,1,x20704,1.988707
4,1,x70128,1.869016
...,...,...,...
69604,1,x69507,-1.397159
69605,1,x103580,-1.583600
69606,1,x166952,-1.627333
69607,1,x67193,-1.811472


In [18]:
def analyze_features(model, n):
  index_to_word = {v:k for k,v in count_vect.vocabulary_.items()}
  data = eli5.formatters.as_dataframe.explain_weights_df(model)
  
  class1 = data[data['target']==1]
  
  print('\n', n, 'самых значимых признаков класса (toxic):')
  for element in class1.feature[:n]:
    feature = element.strip('x')
    print(index_to_word[int(feature)], element)

In [19]:
analyze_features(clf, 30)


 30 самых значимых признаков класса (toxic):
fuck x70089
idiot x84835
faggot x64497
asshole x20704
fucking x70128
idiots x84845
suck x164233
shit x154929
fucked x70099
bitch x28022
stupid x163563
bark x24314
moron x114373
cunt x46526
ass x20589
bastard x24691
nigger x119855
sucks x164261
motherfucker x114630
bullshit x32375
penis x129840
dicks x51942
jerk x91987
cock x40275
dumbass x56450
fuckin x70126
fag x64461
france x69133
nigga x119839
niggers x119867


In [20]:
sum(sum(clf.coef_ != 0))


69608

In [21]:
top_features = [int(i[1:]) for i in eli5.formatters.as_dataframe.explain_weights_df(clf).feature if 'BIAS' not in i]

In [22]:
len(top_features)

69608

### Top features without BIAS

In [24]:
X_train_eli5 = X_train[:,top_features]
X_test_eli5 = X_test[:,top_features]

In [25]:
eli5_model = SGDClassifier().fit(X_train_eli5, y_train)

In [26]:
y_pred = eli5_model.predict(X_test_eli5)

In [27]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.99      0.97     66765
           1       0.81      0.60      0.69      7007

    accuracy                           0.95     73772
   macro avg       0.89      0.79      0.83     73772
weighted avg       0.95      0.95      0.95     73772



По сравнению с бейзлайном precision метки 1 снизился до 0.81. Все остальные метрики не изминились.

### Top 1000 features

In [ ]:
top1000_features = top_features[:1000]

In [ ]:
X_train_eli5 = X_train[:,top1000_features]
X_test_eli5 = X_test[:,top1000_features]

In [ ]:
eli5_model = SGDClassifier().fit(X_train_eli5, y_train)

In [ ]:
y_pred = eli5_model.predict(X_test_eli5)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.96     66765
           1       0.70      0.49      0.58      7007

    accuracy                           0.93     73772
   macro avg       0.82      0.74      0.77     73772
weighted avg       0.92      0.93      0.93     73772



По сравнению с бейзлайном precision метки 1 снизился до 0.70, recall меток 0 и 1 снизился до 0.98 и 0.49, f1-score снизился до 0.93.

### Top 10000 features

In [ ]:
top10000_features = top_features[:10000]

In [ ]:
X_train_eli5 = X_train[:,top10000_features]
X_test_eli5 = X_test[:,top10000_features]

In [ ]:
eli5_model = SGDClassifier().fit(X_train_eli5, y_train)

In [ ]:
y_pred = eli5_model.predict(X_test_eli5)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.97     66765
           1       0.78      0.55      0.65      7007

    accuracy                           0.94     73772
   macro avg       0.87      0.77      0.81     73772
weighted avg       0.94      0.94      0.94     73772



По сравнению с бейзлайном precision меток 0 и 1 снизился до 0.95 0.78, recall меток 0 и 1 снизился до 0.98 и 0.55, f1-score снизился до 0.94. Результаты по сравнению с top 1000 features улучшились. 

### Top 30000 features

In [ ]:
top30000_features = top_features[:30000]

In [ ]:
X_train_eli5 = X_train[:,top30000_features]
X_test_eli5 = X_test[:,top30000_features]

In [ ]:
eli5_model = SGDClassifier().fit(X_train_eli5, y_train)

In [ ]:
y_pred = eli5_model.predict(X_test_eli5)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     66765
           1       0.81      0.54      0.65      7007

    accuracy                           0.94     73772
   macro avg       0.88      0.76      0.81     73772
weighted avg       0.94      0.94      0.94     73772



По сравнению с бейзлайном precision меток 0 и 1 снизился до 0.95 и 0.81, recall метки 1 снизился до 0.54, f1-score снизился до 0.94. Результаты по сравнению с top 10000 features улучшились.

### Top 50000 features

In [ ]:
top50000_features = top_features[:50000]

In [ ]:
X_train_eli5 = X_train[:,top50000_features]
X_test_eli5 = X_test[:,top50000_features]

In [ ]:
eli5_model = SGDClassifier().fit(X_train_eli5, y_train)

In [ ]:
y_pred = eli5_model.predict(X_test_eli5)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     66765
           1       0.82      0.54      0.65      7007

    accuracy                           0.95     73772
   macro avg       0.89      0.77      0.81     73772
weighted avg       0.94      0.95      0.94     73772



По сравнению с бейзлайном precision меток 0 и 1 снизился до 0.95 0.82, recall метки 1 снизился до 0.54. Результаты по сравнению с top 30000 features улучшились.

## 3. Ансамбль из моделей в sklearn (ансамблевые модели типа randomforest не считаются). Нужно минимум 5 разных моделей - 2 балла

In [15]:
from sklearn.utils import resample

data_majority = data[data.toxic==0]
data_minority = data[data.toxic==1]
length = len(data_minority)

data_majority_downsampled = resample(data_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=length,     # to match minority class
                                 random_state=123) # reproducible results

data_downsampled = pd.concat([data_majority_downsampled, data_minority])

In [18]:
data_downsampled.toxic.value_counts()

1    21384
0    21384
Name: toxic, dtype: int64

In [19]:
X = data_downsampled['comment_text']
y = data_downsampled['toxic']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [28]:
clf1 = SGDClassifier(loss = 'hinge')
clf2 = GaussianNB()
clf3 = LogisticRegression()
clf4 = SVC()
clf5 = XGBClassifier()

eclf = VotingClassifier(estimators=[
        ('SGD', clf1), ('GaussianNB', clf2), ('LogReg', clf3),  ('SVC', clf4),  ('XGB', clf5)], voting='soft')

In [29]:
voting = Pipeline([
    ('vect', CountVectorizer(analyzer='word', max_features=200)),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('clf', eclf),
    ])

In [30]:
voting = voting.fit(X_train, y_train)

In [31]:
predictions = voting.predict(X_test)

AttributeError: ignored

In [ ]:
print("Precision: {0:6.2f}".format(precision_score(y_test, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(y_test, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(y_test, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(y_test, predictions)))